In [1]:
import pandas as pd
import numpy as np
import sqlite3
import shutil
import ipynbname
import datetime
import os
import random

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, FunctionTransformer, QuantileTransformer, MultiLabelBinarizer
from sklearn.impute import SimpleImputer

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from tabm_reference import Model, make_parameter_groups
import rtdl_num_embeddings

from Process_Function import RareCategoryTransformer
from Visualization_function import *
import warnings
warnings.filterwarnings(action='ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## 2. Data Load

In [2]:
data_seed = 10

train_path = f'../data/custom_train_{data_seed}.csv'
test_path = f'../data/custom_test_{data_seed}.csv'

# 학습/평가 데이터 로드
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

print(train.shape, test.shape)

(205080, 68) (51271, 67)


In [3]:
def drop_columns(df):
    cols = [
        '불임 원인 - 여성 요인',  # 고유값 1
        '불임 원인 - 정자 면역학적 요인',  # train, test 모두 '1'인 데이터 1개 >> 신뢰할 수 없음
        '난자 해동 경과일',
    ]
    df = df.drop(cols, axis=1)
    return df

def 특정시술유형(train, test):
    def categorize_procedure(proc):
        tokens = [token.strip() for token in proc.split(",") if token.strip() and not token.strip().isdigit()]
        # 우선순위에 따른 범주화
        if tokens.count("Unknown") >= 1:
            return "Unknown"
        if tokens.count("AH") >= 1:
            return "AH"
        if tokens.count("BLASTOCYST") >= 1:
            return "BLASTOCYST"
        if tokens.count("ICSI") >= 2 or tokens.count("IVF") >= 2:
            return "2ICSI_2IVF"
        if tokens.count("IVF") >= 1 and tokens.count("ICSI") >= 1:
            return "IVF_ICSI"
        if tokens == "ICSI":
            return "ICSI"
        if tokens == "IVF":
            return "IVF"
        return ",".join(tokens) if tokens else None

    for df in [train, test]:
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" / ", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(":", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" ", "")

    counts = train['특정 시술 유형'].value_counts()
    allowed_categories = counts[counts >= 100].index.tolist()

    # allowed_categories에 속하지 않는 값은 "Unknown"으로 대체
    train.loc[~train['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"
    test.loc[~test['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"

    train['특정 시술 유형'] = train['특정 시술 유형'].apply(categorize_procedure)
    test['특정 시술 유형'] = test['특정 시술 유형'].apply(categorize_procedure)

    train['시술유형_통합'] = train['시술 유형'].astype(str) + '_' + train['특정 시술 유형'].astype(str)
    test['시술유형_통합'] = test['시술 유형'].astype(str) + '_' + test['특정 시술 유형'].astype(str)

    drop_cols = ['시술 유형', '특정 시술 유형']
    train = train.drop(drop_cols, axis=1)
    test = test.drop(drop_cols, axis=1)

    return train, test

def 시술횟수(df_train):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상':'6회'})
        df_train[col] = df_train[col].str[0].astype(int)
    df_train['시술_임신'] = df_train['총 임신 횟수'] - df_train['총 시술 횟수']
    df_train = df_train.drop('총 시술 횟수', axis=1)
    return df_train

def 배란유도유형(df_train, df_test):
    mapping = {
        '기록되지 않은 시행': 1,
        '알 수 없음': 0,
        '세트로타이드 (억제제)': 0,
        '생식선 자극 호르몬': 0,
    }
    df_train['배란 유도 유형'] = df_train['배란 유도 유형'].replace(mapping)
    df_test['배란 유도 유형'] = df_test['배란 유도 유형'].replace(mapping)
    return df_train, df_test

def 난자기증자나이(df_train, df_test):
    mapping = {
        '만20세 이하': 20,
        '만21-25세': 25,
        '만26-30세': 30,
        '만31-35세': 35,
        '알 수 없음': 20,  # 만20세 이하와 동일하게 처리
    }
    df_train['난자 기증자 나이'] = df_train['난자 기증자 나이'].replace(mapping)
    df_test['난자 기증자 나이'] = df_test['난자 기증자 나이'].replace(mapping)
    return df_train, df_test

def 배아생성주요이유(df_train, df_test):
    df_train['배아 생성 주요 이유'] = df_train['배아 생성 주요 이유'].fillna('DI')
    df_test['배아 생성 주요 이유'] = df_test['배아 생성 주요 이유'].fillna('DI')

    df_train['배아 생성 이유 리스트'] = df_train['배아 생성 주요 이유'].apply(lambda x: [reason.strip() for reason in x.split(',')])
    df_test['배아 생성 이유 리스트'] = df_test['배아 생성 주요 이유'].apply(lambda x: [reason.strip() for reason in x.split(',')])

    mlb = MultiLabelBinarizer()
    train_one_hot = pd.DataFrame(
        mlb.fit_transform(df_train['배아 생성 이유 리스트']),
        columns=mlb.classes_,
        index=df_train.index
    )
    train_one_hot.columns = ['배아생성이유_' + col for col in train_one_hot.columns]

    test_one_hot = pd.DataFrame(
        mlb.transform(df_test['배아 생성 이유 리스트']),
        columns=mlb.classes_,
        index=df_test.index
    )
    test_one_hot.columns = ['배아생성이유_' + col for col in test_one_hot.columns]

    df_train = pd.concat([df_train, train_one_hot], axis=1)
    df_test = pd.concat([df_test, test_one_hot], axis=1)

    cols_to_drop = [
        '배아 생성 주요 이유',
        '배아 생성 이유 리스트',
        '배아생성이유_연구용',
        '배아생성이유_DI'
    ]
    df_train = df_train.drop(cols_to_drop, axis=1, errors='ignore')
    df_test = df_test.drop(cols_to_drop, axis=1, errors='ignore')

    cols = ['배아생성이유_기증용',
            '배아생성이유_난자 저장용',
            '배아생성이유_배아 저장용',
            '배아생성이유_현재 시술용']

    df_train[cols] = df_train[cols].div(df_train[cols].sum(axis=1).replace(0, np.nan), axis=0).fillna(0)
    df_test[cols] = df_test[cols].div(df_test[cols].sum(axis=1).replace(0, np.nan), axis=0).fillna(0)

    return df_train, df_test

def 단일배아이식여부(df_train, df_val):
    df_train['단일 배아 이식 여부'] = df_train['단일 배아 이식 여부'].fillna(0)
    df_val['단일 배아 이식 여부'] = df_val['단일 배아 이식 여부'].fillna(0)
    return df_train, df_val


def 기증자정자와혼합된난자수(df_train, df_test):
    df_train["기증자 정자와 혼합된 난자 수"] = df_train["기증자 정자와 혼합된 난자 수"].fillna(2)
    df_test["기증자 정자와 혼합된 난자 수"] = df_test["기증자 정자와 혼합된 난자 수"].fillna(2)
    return df_train, df_test

def label_encoding(train, test, cols):
    encoder = LabelEncoder()
    for col in cols:
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
    return train, test

def type_to_category(train, test, cols):
    train[cols] = train[cols].astype('category')
    test[cols] = test[cols].astype('category')
    return train, test

def impute_nan(train, test):
    cols_to_impute = [
        '임신 시도 또는 마지막 임신 경과 연수', # DI, IVF랑 관련 X
    ]
    imputer = SimpleImputer(strategy='mean')
    train[cols_to_impute] = imputer.fit_transform(train[cols_to_impute])
    test[cols_to_impute] = imputer.transform(test[cols_to_impute])

    cols_to_impute = [
        '난자 채취 경과일',
        '난자 혼합 경과일',
        '배아 이식 경과일',
        '배아 해동 경과일',

        '착상 전 유전 검사 사용 여부',
        'PGD 시술 여부',
        'PGS 시술 여부',

        ### DI only
        '착상 전 유전 진단 사용 여부',
        '총 생성 배아 수',
        '미세주입된 난자 수',
        '미세주입에서 생성된 배아 수',
        '이식된 배아 수',
        '미세주입 배아 이식 수',
        '저장된 배아 수',
        '미세주입 후 저장된 배아 수',
        '해동된 배아 수',
        '해동 난자 수',
        '수집된 신선 난자 수',
        '저장된 신선 난자 수',
        '혼합된 난자 수',
        '파트너 정자와 혼합된 난자 수',
        '기증자 정자와 혼합된 난자 수',
        '동결 배아 사용 여부',
        '신선 배아 사용 여부',
        '기증 배아 사용 여부',
        '대리모 여부',
        ### DI
    ]
    train[cols_to_impute] = train[cols_to_impute].fillna(0)
    test[cols_to_impute] = test[cols_to_impute].fillna(0)

    return train, test

def num_feature_scailing(train, test, seed=777):
    numeric_cols = train.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    cols_to_scale = [
        col for col in numeric_cols
        if col not in cat_cols and col != '임신 성공 여부'
    ]

    arr_train = train[cols_to_scale].to_numpy()  # DataFrame -> NumPy
    arr_train = arr_train.astype(np.float32)
    arr_test = test[cols_to_scale].to_numpy()
    arr_test = arr_test.astype(np.float32)

    np.random.seed(seed)
    random.seed(seed)
    noise = (
        np.random.default_rng(0)
        .normal(0.0, 1e-5, arr_train.shape)
        .astype(arr_train.dtype)
    )
    preprocessing = QuantileTransformer(
        n_quantiles=max(min(len(train[cols_to_scale]) // 30, 1000), 10),
        output_distribution='normal',
        subsample=10**9,
    ).fit(arr_train + noise)

    # train[cols_to_scale] = preprocessing.transform(arr_train + noise)
    train[cols_to_scale] = preprocessing.transform(arr_train)
    test[cols_to_scale] = preprocessing.transform(arr_test)
    return train, test

def drop_single_value_columns(df_train, df_test):
    cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() == 1]
    return df_train.drop(columns=cols_to_drop), df_test.drop(columns=cols_to_drop)

def all_process(train, val):
    # 기본 전처리 단계
    train, val = drop_columns(train), drop_columns(val)
    train, val = 특정시술유형(train, val)
    train, val = 시술횟수(train), 시술횟수(val)

    train, val = 단일배아이식여부(train, val)
    train, val = 배란유도유형(train, val)
    train, val = 배아생성주요이유(train, val)

    cols_to_encoding = [
        "시술 시기 코드",
        "시술 당시 나이",
        "배란 유도 유형",
        # "클리닉 내 총 시술 횟수",
        # "IVF 시술 횟수",
        # "DI 시술 횟수",
        # "총 임신 횟수",
        # "IVF 임신 횟수",
        # "DI 임신 횟수",
        # "총 출산 횟수",
        # "IVF 출산 횟수",
        # "DI 출산 횟수",
        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",
        '시술유형_통합',
    ]
    train, val = label_encoding(train, val, cols=cols_to_encoding)
    train, val = type_to_category(train, val, cols=cols_to_encoding)

    train, val = impute_nan(train, val)
    train, val = num_feature_scailing(train, val)

    train, val = drop_single_value_columns(train, val)

    return train, val

train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

train, test = all_process(train, test)

cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 여부']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

수치형 변수: 57개 
['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '시술_임신', '배아생성이유_기증용', '배아생성이유_난자 저장용', '배아생성이유_배아 저장용', '배아생성이유_현재 시술용']
범주형 변수: 8개 
['시술 시기 코드', '시술 당시 나이', '배란 유도 유형', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(205080, 66) (51271, 65)


In [4]:
def get_feature_info(train, target_col='임신 성공 여부'):
    n_num_features_ = len(numeric_cols)
    cat_cardinalities_ = [train[col].nunique() for col in cat_cols]

    return n_num_features_, cat_cardinalities_

def to_dataloader(df, batch_size=256, is_shuffle=True, is_train=True, target_col='임신 성공 여부'):
    X_num = torch.tensor(df[numeric_cols].values, dtype=torch.float32)
    X_cat = torch.tensor(df[cat_cols].values, dtype=torch.long)

    if is_train:
        y = torch.tensor(df[target_col].values, dtype=torch.float32)
        tensor_dataset = TensorDataset(X_num, X_cat, y)
        data_loader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=is_shuffle)
    else:
        tensor_dataset = TensorDataset(X_num, X_cat)
        data_loader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=is_shuffle)

    return data_loader

n_num_features, cat_cardinalities = get_feature_info(train)
print(n_num_features)
print(cat_cardinalities)

train_loader = to_dataloader(train)
test_loader = to_dataloader(test, is_shuffle=False, is_train=False)

57
[7, 7, 2, 3, 4, 5, 7, 9]


In [5]:
class TabMWrapper:
    def __init__(self, model_config, trainer_config):
        self.device = trainer_config.get("device") or torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu'
        )
        self.lr = trainer_config.get("lr", 0.001)
        self.weight_decay = trainer_config.get("weight_decay", 3e-4)
        self.criterion = trainer_config.get("criterion", F.cross_entropy)
        self.patience = trainer_config.get("patience", 3)

        # 모델 생성
        self.model = Model(**model_config).to(self.device)

        optimizer_type = trainer_config.get("optimizer", "AdamW")
        params = make_parameter_groups(self.model)
        if optimizer_type == "AdamW":
            self.optimizer = torch.optim.AdamW(params, lr=self.lr, weight_decay=self.weight_decay)
        elif optimizer_type == "Adam":
            self.optimizer = torch.optim.Adam(params, lr=self.lr)
        else:
            raise ValueError(f"Unsupported optimizer: {optimizer_type}")

    def fit(self, train_loader, valid_loader, num_epochs=30, verbose=True):
        train_loss_history = []
        val_loss_history = []

        best_val_loss = float("inf")
        best_epoch = 0
        best_auc = 0

        best_model_state = None
        epochs_without_improvement = 0

        for epoch in range(num_epochs):
            self.model.train()
            epoch_loss = 0.0
            for x_num_batch, x_cat_batch, y_batch in train_loader:
                x_num_batch = x_num_batch.to(self.device)
                # x_cat_batch가 없는 경우에도 대응 (None 체크)
                if x_cat_batch is not None:
                    x_cat_batch = x_cat_batch.to(self.device)
                y_batch = y_batch.to(self.device).long()

                self.optimizer.zero_grad()
                # 모델 출력: (batch, k, ?)
                outputs = self.model(x_num=x_num_batch, x_cat=x_cat_batch)
                # 앙상블 멤버의 예측값 평균 후, 마지막 차원 제거 (예: (B, 1) -> (B,))
                ensemble_logits = outputs.mean(dim=1).squeeze(-1)
                loss = self.criterion(ensemble_logits, y_batch)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item() * x_num_batch.size(0)

            avg_train_loss = epoch_loss / len(train_loader.dataset)
            train_loss_history.append(avg_train_loss)

            avg_val_loss = self.evaluate(valid_loader)
            val_loss_history.append(avg_val_loss)

            if verbose:
                print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

            # Early stopping 체크 (validation loss 기준)
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_model_state = self.model.state_dict()
                epochs_without_improvement = 0
                best_epoch = epoch
                # print("  New best validation loss! Model saved.")
            else:
                epochs_without_improvement += 1
                # print(f"No improvement for {epochs_without_improvement} epoch(s).")

            if epochs_without_improvement >= self.patience:
                break

        # 학습 종료 후, best model의 가중치를 로드
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
            print(f"Best model weights loaded from epoch {best_epoch+1} with validation loss {best_val_loss:.4f}.")

        return {"train_loss_history": train_loss_history, "val_loss_history": val_loss_history}

    def evaluate(self, data_loader):
        # Validation 단계
        self.model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_num_batch, x_cat_batch, y_batch in data_loader:
                x_num_batch = x_num_batch.to(self.device)
                if x_cat_batch is not None:
                    x_cat_batch = x_cat_batch.to(self.device)
                y_batch = y_batch.to(self.device).long()

                outputs = self.model(x_num=x_num_batch, x_cat=x_cat_batch)
                ensemble_logits = outputs.mean(dim=1).squeeze(-1)
                loss = self.criterion(ensemble_logits, y_batch)
                val_loss += loss.item() * x_num_batch.size(0)

        avg_val_loss = val_loss / len(data_loader.dataset)
        return avg_val_loss

    def predict(self, data_loader):
        self.model.eval()
        preds = []
        with torch.no_grad():
            for batch in data_loader:
                # 배치에 포함된 값의 개수를 확인하여 unpack
                if len(batch) == 3:
                    x_num_batch, x_cat_batch, _ = batch
                elif len(batch) == 2:
                    x_num_batch, x_cat_batch = batch
                else:
                    raise ValueError("Unexpected number of values in batch")

                x_num_batch = x_num_batch.to(self.device)
                if x_cat_batch is not None:
                    x_cat_batch = x_cat_batch.to(self.device)
                outputs = self.model(x_num=x_num_batch, x_cat=x_cat_batch)
                ensemble_logits = outputs.mean(dim=1).squeeze(-1)
                probs = torch.softmax(ensemble_logits, dim=1)

                preds_batch = probs.detach().cpu().numpy()
                preds.extend(preds_batch.tolist())
        return np.array(preds)


In [6]:
def get_model_config(n_num_features, cat_cardinalities, bins, arch_type):
    model_config = {
        'n_num_features': n_num_features,
        'cat_cardinalities': cat_cardinalities,
        'n_classes': 2,
        'backbone': {
            'type': 'MLP',
            'n_blocks': 3 if bins is None else 2,
            'd_block': 512,
            'dropout': 0.1,
        },
        'bins': bins,
        'num_embeddings': (
            None if bins is None else {
                'type': 'PiecewiseLinearEmbeddings',
                'd_embedding': 16,
                'activation': False,
                'version': 'B',
            }
        ),
        'arch_type': arch_type,
        'k': 32,
        'share_training_batches': True,
    }
    return model_config


In [7]:
seed_list = [333]
all_auc = []
test_preds = []
train_history = []

for seed in seed_list:
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    auc_scores = []
    fold_test_preds = []

    train = pd.read_csv(train_path).drop(columns=['ID'])
    test = pd.read_csv(test_path).drop(columns=['ID'])

    for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train['임신 성공 여부'])):
        fold_train, fold_valid = train.iloc[train_idx].copy().reset_index(drop=True), train.iloc[valid_idx].copy().reset_index(drop=True)
        fold_train2 = fold_train.copy()
        fold_test = test.copy()

        fold_train, fold_valid = all_process(fold_train, fold_valid)
        fold_train2, fold_test = all_process(fold_train2, fold_test)

        cat_cols = [col for col in fold_train.columns if pd.api.types.is_categorical_dtype(fold_train[col])]
        numeric_cols = [col for col in fold_train.columns if col not in cat_cols and col != '임신 성공 여부']

        # TabM
        # arch_type = 'tabm'
        # bins = None

        # TabM-mini with the piecewise-linear embeddings.
        arch_type = 'tabm-mini'
        bins = rtdl_num_embeddings.compute_bins(torch.tensor(fold_train[numeric_cols].values))
        n_num_features, cat_cardinalities = get_feature_info(fold_train)
        model_config = get_model_config(n_num_features, cat_cardinalities, bins, arch_type)

        trainer_config = {
            'device': None,
            'optimizer': 'AdamW',
            'lr': 2e-3,
            'weight_decay': 3e-4,
            'criterion': F.cross_entropy,
            # 'criterion': F.binary_cross_entropy_with_logits,
            'patience': 3,
        }

        batch_size = 4096
        train_loader = to_dataloader(fold_train, batch_size=batch_size)
        valid_loader = to_dataloader(fold_valid, batch_size=batch_size, is_shuffle=False)
        test_loader = to_dataloader(fold_test, batch_size=batch_size, is_shuffle=False, is_train=False)

        model = TabMWrapper(model_config, trainer_config)
        history = model.fit(train_loader, valid_loader, verbose=False)
        train_history.append(history)

        valid_preds = model.predict(valid_loader)[:, 1]
        fold_auc = roc_auc_score(fold_valid['임신 성공 여부'], valid_preds)
        print(f'Fold {fold + 1} AUC: {fold_auc}')

        auc_scores.append(fold_auc)
        test_pred = model.predict(test_loader)[:, 1]
        fold_test_preds.append(test_pred)

    test_preds.append(np.mean(fold_test_preds, axis=0))

    # 각 seed별 평균 AUC와 표준편차 출력
    seed_auc_mean = np.mean(auc_scores)
    seed_auc_std = np.std(auc_scores)
    all_auc.append(seed_auc_mean)
    print(f"Seed {seed} - Average AUC: {seed_auc_mean:.5f} (STD: {seed_auc_std:.5f})")
    print('=' * 60)



# 전체 결과에 대한 평균 및 표준편차 출력
total_auc_mean = np.mean(all_auc)
total_auc_std = np.std(all_auc)
print('-' * 60)
print(f'Total Average AUC: {total_auc_mean:.6f} (STD: {total_auc_std:.6f})')

Best model weights loaded from epoch 5 with validation loss 0.4887.
Fold 1 AUC: 0.7404446097407157
Best model weights loaded from epoch 6 with validation loss 0.4869.
Fold 2 AUC: 0.741742613602096
Best model weights loaded from epoch 7 with validation loss 0.4909.
Fold 3 AUC: 0.7347143886433024
Best model weights loaded from epoch 8 with validation loss 0.4902.
Fold 4 AUC: 0.7348754487755882
Best model weights loaded from epoch 9 with validation loss 0.4869.
Fold 5 AUC: 0.7404747296792731
Seed 333 - Average AUC: 0.73845 (STD: 0.00302)
------------------------------------------------------------
Total Average AUC: 0.738450 (STD: 0.000000)


In [8]:
old_auc = 0.744533 * 100
old_std = 0.001171 * 100

new_auc = total_auc_mean * 100
new_std = total_auc_std * 100

def calculate_change(old_value, new_value):
    change = new_value - old_value
    percentage_change = (change / old_value) * 100 if old_value != 0 else float('inf')
    return change, percentage_change

def format_change(change):
    return f"{change:+.6f}"

# 각 지표의 변화량 계산
auc_change, auc_pct = calculate_change(old_auc, new_auc)
std_change, std_pct = calculate_change(old_std, new_std)

# 결과 출력
print("\n========== 모델 성능 변화 ==========")
print(f"{'Metric':<8}  {'AUC':>12}  {'Acc':>12}")
print("-" * 36)
print(f"{'Old':<8}  {old_auc:>12.6f}  {old_std:>12.6f}")
print(f"{'New':<8}  {new_auc:>12.6f}  {new_std:>12.6f}")
print(f"{'Change':<8}  {format_change(auc_change):>12}  {format_change(std_change):>12}")
print(f"{'% Change':<8}  {auc_pct:>11.4f}%  {std_pct:>11.4f}%")
print("=" * 36)


========== 모델 성능 변화 ==========
Metric             AUC           Acc
------------------------------------
Old          74.453300      0.117100
New          73.845036      0.000000
Change       -0.608264     -0.117100
% Change      -0.8170%    -100.0000%


In [9]:
tmp_submission = pd.DataFrame({f'tabm_{data_seed}': np.mean(test_preds, axis=0)})
tmp_submission

,tabm_10
0,0.341478
1,0.105108
2,0.000232
3,0.114351
4,0.446093
...,...
51266,0.231313
51267,0.255745
51268,0.141795
51269,0.029850


In [10]:
from LG_Aimers_6th.cal_auc import calculate_auc

score = calculate_auc(tmp_submission, seed=data_seed)
print(f'[seed {data_seed}]: {score}')

[seed 10]: 0.74150757679765
